In [43]:
! pip -q install transformers datasets

In [44]:
! rm -rf memes-dataset.csv
! wget https://github.com/mrsndmn/memes-dataset/raw/main/memes-dataset.csv

--2022-06-17 09:39:10--  https://github.com/mrsndmn/memes-dataset/raw/main/memes-dataset.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrsndmn/memes-dataset/main/memes-dataset.csv [following]
--2022-06-17 09:39:11--  https://raw.githubusercontent.com/mrsndmn/memes-dataset/main/memes-dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4608389 (4.4M) [application/octet-stream]
Saving to: ‘memes-dataset.csv’

memes-dataset.csv   100%[===================>]   4.39M  --.-KB/s    in 0.01s   

2022-06-17 09:39:11 (346 MB/s) - ‘memes-dataset.csv’ saved [4608389/4608389]



In [45]:
from datasets import load_dataset

# Датасет

Описание датасета можно прочитать в нулевой домашке (см ссылку в пине чатика)

In [46]:
!head memes-dataset.csv

template,mem_href,mem_description
110133729/spiderman-pointing-at-spiderman,/i/5whzkb,"
IMAGE DESCRIPTION:
IF YOU HAVE A TWIN YOUR LUCKY "
110133729/spiderman-pointing-at-spiderman,/i/5w2wcr,"
IMAGE DESCRIPTION:
ME; DAD; ME AND MY DAD WHEN MOM SCREAMS WHO ATE THE LAST COOKIE "
110133729/spiderman-pointing-at-spiderman,/i/5uodc5,"
IMAGE DESCRIPTION:
ME; A KID WEARING THE SAME SHIRT I'M WEARING "


In [47]:
dataset = load_dataset('csv', data_files={"train": "memes-dataset.csv"})

Using custom data configuration default-bbf4278dbe98ca5c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bbf4278dbe98ca5c/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
dataset['train']

Dataset({
    features: ['template', 'mem_href', 'mem_description'],
    num_rows: 39315
})

**Задание 1. [0.25 балла]** Переименуйте колонки датасета.  Название колонок должно соответствовать названиями колонок в других датасетах, с которыми работают модели из Transformers: должны быть колонки `text` и `label`

Используйте `dataset.rename_columns`.

**Задание 2. [0.25 балла]** Отфильтруйте из датасета строчки, в которых нет текста.

In [49]:
dataset = dataset.rename_columns({'template': 'label', 'mem_description': 'text'})# rename
dataset = dataset.filter(lambda x: x['text']!=None) # filter

dataset

  0%|          | 0/40 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'mem_href', 'text'],
        num_rows: 38783
    })
})

In [50]:
dataset['train'][0]

{'label': '110133729/spiderman-pointing-at-spiderman',
 'mem_href': '/i/5whzkb',
 'text': '\nIMAGE DESCRIPTION:\nIF YOU HAVE A TWIN YOUR LUCKY '}

**Задание 3. [0.25 балла]** Заметили, что каждый текст наинается с одного и того же префикса? Давайте обрежим этот префикс. Испоьлзуйте `dataset.map`

In [51]:
dataset = dataset.map(lambda x: {'text': x['text'][20:]}) # map

  0%|          | 0/38783 [00:00<?, ?ex/s]

In [52]:
dataset['train'][0]

{'label': '110133729/spiderman-pointing-at-spiderman',
 'mem_href': '/i/5whzkb',
 'text': 'IF YOU HAVE A TWIN YOUR LUCKY '}

**Задание 4. [0.25 балла]** Закодируйте метки датасета с помощью LabelEncoder и `dataset.map`.

Для того, чтобы получить список уникальных лейблов можно, например, превратить датасет сначала в датафрейм: `dataset['train'].to_pandas()`. А как получить из колонки датафрейма уникальные значения мы знаем.

In [53]:
from sklearn.preprocessing import LabelEncoder

In [54]:
templates = dataset['train'].to_pandas()['label'].unique() # get uniq templates
templates[:3]

array(['110133729/spiderman-pointing-at-spiderman',
       '113387478/The-Cooler-Daniel',
       '133946291/You-know-Im-something-of-a-scientist-myself'],
      dtype=object)

In [55]:
le = LabelEncoder()# label encoder + fit
le.fit(templates)

LabelEncoder()

In [56]:
import numpy as np
dataset = dataset.map(lambda x: {'label': int(le.transform(np.array(x['label']).reshape(1)))}) # map dataset

  0%|          | 0/38783 [00:00<?, ?ex/s]

# Токенизация

Нужно токенизировать датасет. В качестве модели (в том числе чеккпоинт для токенайзера) используйте https://huggingface.co/distilbert-base-uncased

Важно посмотреть на результат токенизации: не будет ли там ничего лишнего?

**Задание 5. [0.25 балла]** Загрузите токенизатор и токенизируйте датасет

Разобъем датасет на трейн/тест

Эту ячейку менять нельзя

In [57]:
dataset = dataset['train'].train_test_split(test_size=0.05, seed=42)

In [58]:
from transformers import AutoTokenizer

MODEL_CHECKPOINT = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT) # load tokenizer

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

**Задание 6. [0.25 балла]** Смотрим, как работает токенизатор

Проведите токенизацию первого предложения в тексте. Напечатайте исходное предложение, потом выведете список токенов (это "массив" чисел), потом сделайте обратное преобразование списка токенов с помощью `tokenizer.decode`

In [59]:
text = tokenized_datasets['train'][0]['text']  # get text
print(text)

inputs_ids = tokenizer(text)['input_ids'] # get tokenized text
print(inputs_ids)

detokenized = tokenizer.decode(inputs_ids) # decode tokenized text
print(detokenized)

RICK TELLING MORTY ABOUT ALL THE OTHER ADVENTURES THAT THE NEED TO GO ON 
[101, 6174, 4129, 22294, 2100, 2055, 2035, 1996, 2060, 7357, 2008, 1996, 2342, 2000, 2175, 2006, 102]
[CLS] rick telling morty about all the other adventures that the need to go on [SEP]


In [60]:
tokenized_datasets['train'][0]['label']#it was 0

42

In [61]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'mem_href', 'text', 'input_ids', 'attention_mask'],
        num_rows: 36843
    })
    test: Dataset({
        features: ['label', 'mem_href', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1940
    })
})

Загрузите модель из того же чекпонта, что и токенайзер

# Модель

In [62]:
from transformers import AutoModelForSequenceClassification

num_labels = max(tokenized_datasets['train']['label'])+1 #

model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

# Обучение

**Задание 7. [2 балла]** Обучите модель так, чтобы accuracy на валидационном датасете было не меньше `0.4`

Обратите внимание на `batch_size`, `learning_rate`, другие гиперпараметры обучения. Можно дообработать датасет, если вы считаете это необходимым. Возможно, трейн несбалансированный, возможно, он слишком большой

In [63]:
from transformers import TrainingArguments

In [64]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [65]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    seed=42,
    learning_rate=1e-4,
    num_train_epochs=2
)

In [66]:
#Есть дисбаланс классов, поэтому далее воспользуемся взвешенным лоссом при обучении
from collections import Counter
Counter(dataset['train'].to_pandas()['label'])

Counter({0: 416,
         1: 65,
         2: 61,
         3: 28,
         4: 43,
         5: 123,
         6: 383,
         7: 112,
         8: 23,
         9: 124,
         10: 334,
         11: 43,
         12: 1266,
         13: 359,
         14: 43,
         15: 1138,
         16: 100,
         17: 1292,
         18: 1293,
         19: 305,
         20: 1253,
         21: 1276,
         22: 1282,
         23: 1272,
         24: 1193,
         25: 1271,
         26: 1281,
         27: 1279,
         28: 1272,
         29: 1274,
         30: 1279,
         31: 1283,
         32: 1285,
         33: 1269,
         34: 1301,
         35: 1272,
         36: 1299,
         37: 1295,
         38: 1277,
         39: 1253,
         40: 1296,
         41: 1261,
         42: 1269})

In [67]:
#Основная часть кода взята из: https://discuss.huggingface.co/t/how-can-i-use-class-weights-when-training/1067/6
import torch
from torch import nn
from transformers import Trainer
from sklearn.utils.class_weight import compute_class_weight

class_weights=compute_class_weight(class_weight='balanced',classes=dataset['train'].to_pandas()['label'].unique(),y=dataset['train'].to_pandas()['label'])
class_weights=torch.tensor(class_weights,dtype=torch.float).cuda()

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [68]:
# если возникает ошибка с CUDA можно попробовать установить эту переменную изи воспроизвести ошибку в колабе в среде без GPU
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

trainer = CustomTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [69]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, mem_href. If text, mem_href are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36843
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4606


Epoch,Training Loss,Validation Loss,Accuracy
1,1.364500,1.397620,0.354639
2,0.888500,1.353482,0.455670


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=4606, training_loss=1.247715726811835, metrics={'train_runtime': 3289.0931, 'train_samples_per_second': 22.403, 'train_steps_per_second': 1.4, 'total_flos': 9768129748752384.0, 'train_loss': 1.247715726811835, 'epoch': 2.0})

**Задание 8. [0.25 балла]**  Какой `data_collator` используется в процессе обучения? Зачем он нужен? Покажите, как он работает?


**Ответ:** (ожидается текстовый ответ на первые 2 вопроса и ячейка, которая демонстрирует работу `data_collator`'а)

Используется DataCollatorWithPadding. data_collator нужен для подгрузки батчей из обучающего датасета для непосредственно обучения.

In [70]:
data_collator = trainer.data_collator
type(data_collator)

transformers.data.data_collator.DataCollatorWithPadding

In [71]:
text = tokenized_datasets['train'][0:2]['text']  # get text
inputs_ids = tokenizer(text)['input_ids']

batch = {'input_ids': inputs_ids}
data_collator(batch)

{'input_ids': tensor([[  101,  6174,  4129, 22294,  2100,  2055,  2035,  1996,  2060,  7357,
          2008,  1996,  2342,  2000,  2175,  2006,   102,     0,     0],
        [  101,  2111,  2006, 10047, 25708, 15000,  1025,  4567,  2033,  7834,
          1025,  2003,  2023,  1037,  2919,  2033,  4168,  1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

# Тестирование

**Задание 9. [0.25 балла]**  Создайте text-classification pipeline, который будет использовать вашу обученную модель и токеннайзер, протестируйте вашу модель

In [76]:
from transformers import pipeline

classification_pipeline = pipeline('text-classification', model=model, tokenizer=tokenizer,device=0)

In [77]:
classification_pipeline("i bet hes thinking about me")

[{'label': 'LABEL_30', 'score': 0.9984914064407349}]

In [78]:
le.inverse_transform([30])

array(['I-Bet-Hes-Thinking-About-Other-Women'], dtype=object)